In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [2]:
def get_shape(tensor):
    "return the shape of tensor as list"
    return tensor.get_shape().as_list()

def print_var(string):
    "print var_name and var_shape in collection named string"
    print("\n".join(["{} : {}".format(v.name, get_shape(v)) for v in tf.get_collection(string)]))
    
def clip(x, vmax = 1-1e-7, vmin = 1e-7):
    "clip the x between vmax, vmin"
    return tf.clip_by_value(x, clip_value_max=vmax, clip_value_min=vmin)

In [3]:
mnist = input_data.read_data_sets('../MNIST_data', one_hot=True)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
train = {}
test = {}
val = {}
train['input'] = mnist.train.images
train['output'] = mnist.train.labels
test['input'] = mnist.test.images
test['output'] = mnist.test.labels
val['input'] = mnist.validation.images
val['output'] = mnist.validation.labels

In [5]:
# Model parameter
image_size = 28*28
hidden = 300
nclasses = 10

# Train parameter
learning_rate = 1e-4
epoch = 5
ntrain = len(train['input'])
batch_size = 100
nbatch = ntrain//batch_size
log_every = 50

In [6]:
graph = tf.get_default_graph()

In [7]:

with tf.name_scope('input'):
    x = tf.placeholder(dtype = tf.float32, shape = [None, image_size], name = 'image')
    y = tf.placeholder(dtype = tf.float32, shape = [None, nclasses], name = 'label')

layer1 = {}
layer2 = {}    
    
with tf.name_scope("layer1"):
    layer1['w'] = tf.Variable(initial_value = tf.random_normal(shape = [image_size, hidden], mean = 0.0, stddev = 1.0),
                              name = 'weight')
    layer1['b'] = tf.Variable(initial_value = tf.constant(value=0.0, shape = [hidden]),
                              name = 'bias')
    layer1['output'] = tf.nn.sigmoid(tf.add(tf.matmul(x, layer1['w']), layer1['b']))

with tf.name_scope("layer2"):
    layer2['w'] = tf.Variable(initial_value = tf.random_normal(shape = [hidden, nclasses], mean = 0.0, stddev = 1.0),
                              name = 'weight')
    layer2['b'] = tf.Variable(initial_value = tf.constant(value=0.0, shape = [nclasses]),
                              name = 'bias')
    layer2['output'] = tf.nn.softmax(tf.add(tf.matmul(layer1['output'], layer2['w']), layer2['b']))
    
with tf.name_scope('cross_entropy'):
    cross_entropy = -tf.reduce_mean(y*tf.log(clip(layer2['output'])))

with tf.name_scope('train'):
    run_train = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(layer2['output'], 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

pre_summary = [v for v in graph.get_operations()]
tf.summary.scalar("cost_track", cross_entropy)
tf.summary.scalar("accuracy", accuracy)

tf.summary.histogram("weights1_hist", layer1['w'])
tf.summary.histogram("biases1_hist", layer1['b'])

tf.summary.histogram("weights2_hist", layer2['w'])
tf.summary.histogram("biases2_hist", layer2['b'])

summary_op = tf.summary.merge_all()
post_summary = [v for v in graph.get_operations()]

In [8]:
print_var("trainable_variables")

layer1/weight:0 : [784, 300]
layer1/bias:0 : [300]
layer2/weight:0 : [300, 10]
layer2/bias:0 : [10]


In [9]:
[v.name for v in post_summary  if v not in pre_summary]

['cost_track/tags',
 'cost_track',
 'accuracy/tags',
 'accuracy',
 'weights1_hist/tag',
 'weights1_hist',
 'biases1_hist/tag',
 'biases1_hist',
 'weights2_hist/tag',
 'weights2_hist',
 'biases2_hist/tag',
 'biases2_hist',
 'Merge/MergeSummary']

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
logs_path = "./log"
writer = tf.summary.FileWriter(logs_path, sess.graph)

for epoch_ in range(epoch):
    index = np.arange(len(train['input']))
    np.random.shuffle(index)
    trX = train['input'][index]
    trY = train['output'][index]
    
    for nbatch_ in range(nbatch):
        train_feed_dict = {x : trX[batch_size*nbatch_:batch_size*(nbatch_+1)], 
                           y : trY[batch_size*nbatch_:batch_size*(nbatch_+1)]}
        # sess.run
        _, summary = sess.run([run_train, summary_op], feed_dict=train_feed_dict)
        # wrtie summary
        writer.add_summary(summary, epoch_ * nbatch+ nbatch_)

In [11]:
summary

b'\n\x11\n\ncost_track\x15\x9b\xe3\x1c>\n\x0f\n\x08accuracy\x15\\\x8fB?\n\x85 \n\rweights1_hist*\xf3\x1f\t\x00\x00\x00\x00.u\x11\xc0\x11\x00\x00\x00`\xcfM\x12@\x19\x00\x00\x00\x00\x00\xb6\x0cA!`f;\xddZ\xe3y@)\x18\xd7EV\x15\x90\x0cA2\xe0\x0f!\xd1\x0e\x9av\x8b\x12\xc0{2\xf6.\xe0\xdb\x10\xc0\x81DK\xf8\x0b\xa7\x0e\xc0u\xf8r\xca\xad\xdd\x0b\xc0\r\x9c\xc5\xa0)U\t\xc0h\xbc\x105\x9a\x07\x07\xc0\xea6\x0fv\xa3\xef\x04\xc0w`<\x0ef\x08\x03\xc0\xe0\x11\xf1\x0ctM\x01\xc0\xdc\x94\xcd.\x8du\xff\xbfS\x87F\x13i\x99\xfc\xbfyL\xfa\xe2\xd3\xff\xf9\xbf\x85E\xcc\x9f\xc0\xa2\xf7\xbf3?\xe8\x1c\xaf|\xf5\xbf\xbaP\x8d1\x9f\x88\xf3\xbfcI\xc6\x15\x05\xc2\xf1\xbf+\x14\xfa\xcd\xbe$\xf0\xbf\x08\x99i\x19\x15Z\xed\xbf\x1e\x8b\x8e\xe8\xfb\xae\xea\xbf2g\xc7G\xfcA\xe8\xbf\x16uo\xfbp\r\xe6\xbf+Se*8\x0c\xe4\xbf\x0f\x1d\\l\xa79\xe2\xbf\xb01%\xee\x80\x91\xe0\xbf\xfa\xe5Z%\xd3\x1f\xde\xbfW\xe8i\xf3\xbfb\xdb\xbf\xda^\xbd\xaeh\xe5\xd8\xbf\x97\xca7\x13\x02\xa2\xd6\xbf\xcfCa\xb4G\x93\xd4\xbf_&A\xbbo\xb4\xd2\xbf\x9c\x0b\x81\xc1\x1f\

In [12]:
tf.get_default_graph()

In [13]:
sess.graph